# Flight Price Web Scrapping:

ANKUSH C Intership Batch:25

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import selenium.webdriver
from datetime import date
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from dateutil.relativedelta import relativedelta
import csv
import linecache
import sys

In [2]:
def PrintException():
    exc_type, exc_obj, tb = sys.exc_info()
    f = tb.tb_frame
    lineno = tb.tb_lineno
    filename = f.f_code.co_filename
    linecache.checkcache(filename)
    line = linecache.getline(filename, lineno, f.f_globals)
    print('EXCEPTION IN ({}, LINE {} "{}"): {}'.format(filename, lineno, line.strip(), exc_obj))

In [3]:
def scrap(origin,destin,trDate):
    global none
    baseDataUrl = "https://www.makemytrip.com/flight/search?itinerary="+ origin +"-"+ destin +"-"+ trDate +"&tripType=O&paxType=A-1_C-0_I-0&intl=false&=&cabinClass=E"

    try:
        driver = selenium.webdriver.Chrome(r"C:\Users\POOJA C\Downloads\chromedriver_win32\chromedriver.exe") 
        print ("Requesting URL: " + baseDataUrl)

        driver.get(baseDataUrl)  
        print ("Webpage found ...")

        element_xpath = '//*[@id="left-side--wrapper"]/div[2]'

        
        element = WebDriverWait(driver, 15).until(EC.visibility_of_element_located((By.XPATH, element_xpath)))

        print ("Scrolling document upto bottom ...")
        for j in range(1, 100):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        
        body = driver.find_element_by_tag_name("body").get_attribute("innerHTML")

        print("Closing Chrome ...")
        driver.quit()

        print("Getting data from DOM ...")
        soupBody = BeautifulSoup(body)
        
        spanFlightName = soupBody.find_all("span", 'boldFont blackText airlineName')
        print("spanFlightName  = " + str(len(spanFlightName)))

        Time = soupBody.find_all("p", {"class":"blackText fontSize18 blackFont appendBottom2 makeFlex hrtlCenter"})			# Tags with Departure Time
        print("Time Count in Scraping  = " + str(len(Time)))

        City = soupBody.findAll("p", {"class": "darkText"})
        print("City Count in Scraping  = " + str(len(City)))

        spanFlightCost = soupBody.findAll("p", {"class": "blackText fontSize18 blackFont white-space-no-wrap"})			# Tags with Flight Cost
        print("spanFlightCost Count in Scraping  = " + str(len(spanFlightCost)))

        pFlightStops = soupBody.findAll("p", {"class": "flightsLayoverInfo"})
        print("pFlightStops Count in Scraping  = " + str(len(pFlightStops)))
        
        transform(spanFlightName,trDate,City,Time,spanFlightCost,pFlightStops)



   
    except :
        PrintException()
        print(trDate," scraping complete")
        
        none =True 

In [4]:
dates=(pd.date_range(start='24/11/2021', end='30/11/2021'))
New_date=[]
for i,date in enumerate(dates):
    day=str(date.day).zfill(2)   
    month=str(date.month).zfill(2)
    year=str(date.year)
    date=day+'/'+month+'/'+year  
    New_date.append(date)
print(New_date)

['24/11/2021', '25/11/2021', '26/11/2021', '27/11/2021', '28/11/2021', '29/11/2021', '30/11/2021']


In [5]:
#Lists to store scraped data
Airline=[]
Journey_date=[]
From=[]
To=[]
Price=[]
Dtime=[]
Atime=[]
Stops=[]

flight_data = pd.DataFrame(np.nan, index=[0], columns=['Airline', 'Journey_date','From','To','Dtime','Atime','Stops','Price'])
flight_data.to_csv('Flight.csv')
 
def transform(spanFlightName,trDate,City,Time,spanFlightCost,pFlightStops):
    
    Airline.clear()
    Journey_date.clear()
    Price.clear()
    Stops.clear()
    From.clear()
    Dtime.clear()
    To.clear()
    Atime.clear()
    
    global flight_data
    print("Transaform Function Running.... ")
    for i in range(0,len(spanFlightName)):
        
        Airline.append(spanFlightName[i].string)
        Journey_date.append(trDate)
        Price.append(spanFlightCost[i].string)
        Stops.append(pFlightStops[i].string)
        
      
    for j in range(0,len(Time)):
        
        if(j%2==0):
            
            From.append(City[j].string)
            Dtime.append(Time[j].string)
        else:
            
            To.append(City[j].string)
            Atime.append(Time[j].string) 

            
            
    temp = {'Airline':Airline,
            'Journey_date':Journey_date,
            'From':From,
            'To':To,
            'Dtime':Dtime,
            'Atime':Atime,
            'Stops':Stops,
            'Price':Price}
    df=pd.DataFrame(temp)
    df.to_csv('Flight.csv', mode='a', header=False)
    flight_data = pd.concat([flight_data,df])
    flight_data=flight_data.reset_index(drop=True)
    display(flight_data)
    print('Lenght : ',len(flight_data))

In [6]:
list_of_dest=['GOI','AMD','TIR','BLR','BOM','DEL']
for i in range(0,len(list_of_dest)):
    orgin=list_of_dest[i]
    for_to=list_of_dest
    print("--------------")
    for j in range(0,len(for_to)):
        destin=for_to[j]
        none = False
        print("origin = "+orgin)
        print("destin = "+destin+'\n')
        if orgin!=destin:
            for i,date in enumerate(New_date) :
                if none !=True:
                    scrap(orgin,destin,date)
                    print(" ")
                else:
                    break
            else: 
                print( "-----Skiped-----")
                print(" ")

--------------
origin = GOI
destin = GOI

origin = GOI
destin = AMD

Requesting URL: https://www.makemytrip.com/flight/search?itinerary=GOI-AMD-24/11/2021&tripType=O&paxType=A-1_C-0_I-0&intl=false&=&cabinClass=E
Webpage found ...
Scrolling document upto bottom ...
Closing Chrome ...
Getting data from DOM ...
spanFlightName  = 23
Time Count in Scraping  = 46
City Count in Scraping  = 46
spanFlightCost Count in Scraping  = 23
pFlightStops Count in Scraping  = 23
Transaform Function Running.... 


,Airline,Journey_date,From,To,Dtime,Atime,Stops,Price
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Spicejet,24/11/2021,Goa,Ahmedabad,08:15,10:25,Non stop,"₹ 5,315"
2,Spicejet,24/11/2021,Goa,Ahmedabad,21:05,23:00,Non stop,"₹ 5,315"
3,Go First,24/11/2021,Goa,Ahmedabad,14:00,18:35,1 stop via Bengaluru,"₹ 5,315"
4,Go First,24/11/2021,Goa,Ahmedabad,13:15,20:25,1 stop via Mumbai,"₹ 5,315"
5,Go First,24/11/2021,Goa,Ahmedabad,06:40,20:25,1 stop via Mumbai,"₹ 5,315"
6,Go First,24/11/2021,Goa,Ahmedabad,18:15,00:05,1 stop via Mumbai,"₹ 5,315"
7,Go First,24/11/2021,Goa,Ahmedabad,17:00,00:05,1 stop via Mumbai,"₹ 5,315"
8,Go First,24/11/2021,Goa,Ahmedabad,23:45,07:30,1 stop via New Delhi,"₹ 5,315"
9,Go First,24/11/2021,Goa,Ahmedabad,17:40,07:30,1 stop via New Delhi,"₹ 5,315"


Lenght :  24
 
Requesting URL: https://www.makemytrip.com/flight/search?itinerary=GOI-AMD-25/11/2021&tripType=O&paxType=A-1_C-0_I-0&intl=false&=&cabinClass=E
Webpage found ...
Scrolling document upto bottom ...
Closing Chrome ...
Getting data from DOM ...
spanFlightName  = 24
Time Count in Scraping  = 48
City Count in Scraping  = 50
spanFlightCost Count in Scraping  = 24
pFlightStops Count in Scraping  = 24
Transaform Function Running.... 


,Airline,Journey_date,From,To,Dtime,Atime,Stops,Price
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Spicejet,24/11/2021,Goa,Ahmedabad,08:15,10:25,Non stop,"₹ 5,315"
2,Spicejet,24/11/2021,Goa,Ahmedabad,21:05,23:00,Non stop,"₹ 5,315"
3,Go First,24/11/2021,Goa,Ahmedabad,14:00,18:35,1 stop via Bengaluru,"₹ 5,315"
4,Go First,24/11/2021,Goa,Ahmedabad,13:15,20:25,1 stop via Mumbai,"₹ 5,315"
5,Go First,24/11/2021,Goa,Ahmedabad,06:40,20:25,1 stop via Mumbai,"₹ 5,315"
6,Go First,24/11/2021,Goa,Ahmedabad,18:15,00:05,1 stop via Mumbai,"₹ 5,315"
7,Go First,24/11/2021,Goa,Ahmedabad,17:00,00:05,1 stop via Mumbai,"₹ 5,315"
8,Go First,24/11/2021,Goa,Ahmedabad,23:45,07:30,1 stop via New Delhi,"₹ 5,315"
9,Go First,24/11/2021,Goa,Ahmedabad,17:40,07:30,1 stop via New Delhi,"₹ 5,315"


Lenght :  48
 
Requesting URL: https://www.makemytrip.com/flight/search?itinerary=GOI-AMD-26/11/2021&tripType=O&paxType=A-1_C-0_I-0&intl=false&=&cabinClass=E
Webpage found ...
Scrolling document upto bottom ...
Closing Chrome ...
Getting data from DOM ...
spanFlightName  = 22
Time Count in Scraping  = 44
City Count in Scraping  = 46
spanFlightCost Count in Scraping  = 22
pFlightStops Count in Scraping  = 22
Transaform Function Running.... 


...

Lenght :  5091
 
Requesting URL: https://www.makemytrip.com/flight/search?itinerary=DEL-BOM-29/11/2021&tripType=O&paxType=A-1_C-0_I-0&intl=false&=&cabinClass=E
Webpage found ...
Scrolling document upto bottom ...
Closing Chrome ...
Getting data from DOM ...
spanFlightName  = 56
Time Count in Scraping  = 112
City Count in Scraping  = 114
spanFlightCost Count in Scraping  = 56
pFlightStops Count in Scraping  = 56
Transaform Function Running.... 


,Airline,Journey_date,From,To,Dtime,Atime,Stops,Price
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Spicejet,24/11/2021,Goa,Ahmedabad,08:15,10:25,Non stop,"₹ 5,315"
2,Spicejet,24/11/2021,Goa,Ahmedabad,21:05,23:00,Non stop,"₹ 5,315"
3,Go First,24/11/2021,Goa,Ahmedabad,14:00,18:35,1 stop via Bengaluru,"₹ 5,315"
4,Go First,24/11/2021,Goa,Ahmedabad,13:15,20:25,1 stop via Mumbai,"₹ 5,315"
...,...,...,...,...,...,...,...,...
5142,Air India,29/11/2021,New Delhi,Mumbai,19:00,21:15,Non stop,"₹ 5,955"
5143,Spicejet,29/11/2021,New Delhi,Mumbai,19:45,22:05,Non stop,"₹ 5,955"
5144,Vistara,29/11/2021,New Delhi,Mumbai,19:50,22:00,Non stop,"₹ 5,955"
5145,IndiGo,29/11/2021,New Delhi,Mumbai,20:00,22:15,Non stop,"₹ 5,955"


Lenght :  5147
 
Requesting URL: https://www.makemytrip.com/flight/search?itinerary=DEL-BOM-30/11/2021&tripType=O&paxType=A-1_C-0_I-0&intl=false&=&cabinClass=E
Webpage found ...
Scrolling document upto bottom ...
Closing Chrome ...
Getting data from DOM ...
spanFlightName  = 57
Time Count in Scraping  = 114
City Count in Scraping  = 116
spanFlightCost Count in Scraping  = 57
pFlightStops Count in Scraping  = 57
Transaform Function Running.... 


,Airline,Journey_date,From,To,Dtime,Atime,Stops,Price
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Spicejet,24/11/2021,Goa,Ahmedabad,08:15,10:25,Non stop,"₹ 5,315"
2,Spicejet,24/11/2021,Goa,Ahmedabad,21:05,23:00,Non stop,"₹ 5,315"
3,Go First,24/11/2021,Goa,Ahmedabad,14:00,18:35,1 stop via Bengaluru,"₹ 5,315"
4,Go First,24/11/2021,Goa,Ahmedabad,13:15,20:25,1 stop via Mumbai,"₹ 5,315"
...,...,...,...,...,...,...,...,...
5199,Air India,30/11/2021,New Delhi,Mumbai,19:00,21:15,Non stop,"₹ 5,955"
5200,Spicejet,30/11/2021,New Delhi,Mumbai,19:45,22:05,Non stop,"₹ 5,955"
5201,Vistara,30/11/2021,New Delhi,Mumbai,19:50,22:00,Non stop,"₹ 5,955"
5202,IndiGo,30/11/2021,New Delhi,Mumbai,21:15,23:30,Non stop,"₹ 5,955"


Lenght :  5204
 
-----Skiped-----
 
origin = DEL
destin = DEL



In [7]:
#checking scraped Flight Price data
df=pd.read_csv("Flight.csv")
df

,Unnamed: 0,Airline,Journey_date,From,To,Dtime,Atime,Stops,Price
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,Spicejet,24/11/2021,Goa,Ahmedabad,08:15,10:25,Non stop,"₹ 5,315"
2,1,Spicejet,24/11/2021,Goa,Ahmedabad,21:05,23:00,Non stop,"₹ 5,315"
3,2,Go First,24/11/2021,Goa,Ahmedabad,14:00,18:35,1 stop via Bengaluru,"₹ 5,315"
4,3,Go First,24/11/2021,Goa,Ahmedabad,13:15,20:25,1 stop via Mumbai,"₹ 5,315"
...,...,...,...,...,...,...,...,...,...
5199,52,Air India,30/11/2021,New Delhi,Mumbai,19:00,21:15,Non stop,"₹ 5,955"
5200,53,Spicejet,30/11/2021,New Delhi,Mumbai,19:45,22:05,Non stop,"₹ 5,955"
5201,54,Vistara,30/11/2021,New Delhi,Mumbai,19:50,22:00,Non stop,"₹ 5,955"
5202,55,IndiGo,30/11/2021,New Delhi,Mumbai,21:15,23:30,Non stop,"₹ 5,955"


In [17]:
#saving Flight Price data as csv file
df.to_csv("flight_price.csv")

# Successfully Completed scrapping:)